In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/jasome-fastapi'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/jasome-fastapi


In [2]:
import csv

In [3]:
f = open('./dataset/selfs_meta.csv', 'r')
rdr = csv.reader(f)
data = list(rdr)
print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
import re

In [5]:
def process(data, file_name):
  result = ''
  train = ''
  print(len(data))
  print(int(len(data)*0.9))
  try:
    for i, (company, field, spec, question, content) in enumerate(data[1:]):
      if re.match(r".*\(.*\)",company):
        # 회사명에 (주), (학) 등 제거
        company = re.sub(r"\(.*\)","", company)
      company = company.replace("㈜", "")
      # 시작과 끝에 [" "] 제거
      content = re.sub(r"^\[[\'\"]","", content)
      content = re.sub(r"[\'\"]\]$","", content) 
      # 글자수 ~~ byte 제거 
      content = re.sub(r"\\n글자수.*Byte", "", content) 
      # 소제목은 대부분 처음에 "로 바로 시작하고 줄바꿈 두 번 하는 특성이 있음
      content = re.sub(r"\".*?\"\\n\\n", "", content)
      content = re.sub(r"\'.*?\'\\n\\n", "", content)
      content = re.sub(r"\[.*?\]\\n\\n", "", content)
      # 두번 줄바꿈 한번으로
      content = re.sub(r"\\n\\n", "\n", content) 
      # \n을 진짜 개행문자로 변환
      content = content.replace(r"\n", "\n")
      # 회사 이름 제거
      content = content.replace(company, "<귀사>")
      # . 를 .\n으로 변환
      content = re.sub(r"(?<=\.) ", "\n", content)
      content = re.sub(r"(?<=\.)\n", "\n", content)

      # 잡코리아에서 추가된 문자열 제거
      # 아쉬운점 제거
      content = re.sub(r"아쉬운점 \d", "", content)
      # 좋은점 제거
      content = re.sub(r"좋은점 \d", "", content)

      # 공백 제거
      content.strip()
      if content=="" or content.isspace():
        continue
      content = content.replace('[', " ")
      content = content.replace(']', " ")
      
      if i < int(len(data)*0.9):
        result += content + '\n'
      else:
        train += content + '\n'
  except ValueError as e:
    pass
  output = open("../after/train/{}_train.txt".format(file_name), 'w')
  output.write(result)
  output.close()
  output2 = open("../after/test/{}_test.txt".format(file_name), 'w')
  output2.write(train)
  output2.close()


In [6]:
import os

In [7]:
!pwd

/content/drive/MyDrive/Colab Notebooks/jasome-fastapi


In [8]:
print(os.getcwd())
os.chdir('./before')
print(os.listdir())

/content/drive/MyDrive/Colab Notebooks/jasome-fastapi
['selfs_sales_meta.csv', 'selfs_businees_meta.csv', 'selfs_marketing_meta.csv', 'selfs_produce_meta.csv', 'selfs_it_meta.csv']


In [9]:
for original_file_name in os.listdir():
  f = open(original_file_name, 'r')
  rdr = csv.reader(f)
  data = list(rdr)
  f.close()
  print(os.path.splitext(original_file_name)[0])
  process(data, os.path.splitext(original_file_name)[0])


selfs_sales_meta
1218
1096
selfs_businees_meta
1557
1401
selfs_marketing_meta
881
792
selfs_produce_meta
1117
1005
selfs_it_meta
1010
909


In [ ]:
!pip install jpype1
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
company_names = []
for word, pos in kkma.pos('우리에프아이에스'):
  print(word, pos)
  if pos == 'NNP':
    company_names.append(word)
print(company_names)


우리 NP
에프 NNG
아이 NNG
에 JKM
스 VV
[]
